In [ ]:
from utils.game_state import GameState
from utils.snake import Snake
from utils.vector import Vector, up, down, left, right, noop, directions

from logic.bad_moves import BadMoves
from logic.chaise_tail import ChaiseTail
from logic.eat import Eat
from logic.kill import Kill
from logic.orthogonal_distances import OrthogonalDistances
from logic.path_distances import PathDistances
from logic.increase_board_control import IncreaseBoardControl
from logic.surround import Surround

from src.floodfill import is_coords_open, calc_neighbors, calc_open_space
from src.pathfinding import calc_possible_moves, calc_next_move
from src.pathfinding import BattlesnakeAStarPathfinder
from src.targeting import calc_targets
from src.util import calc_manhattan_distance

from time import perf_counter

In [2]:
data = {
    "game": {
        "id": "game-00fe20da-94ad-11ea-bb37",
        "ruleset": {
            "name": "standard",
            "version": "v.1.2.3",
            "settings": {
                "foodSpawnChance": 25,
                "minimumFood": 1,
                "hazardDamagePerTurn": 14,
                "hazardMap": "hz_spiral",
                "hazardMapAuthor": "altersaddle",
                "royale": {"shrinkEveryNTurns": 5},
                "squad": {
                    "allowBodyCollisions": True,
                    "sharedElimination": True,
                    "sharedHealth": True,
                    "sharedLength": True,
                },
            },
        },
        "source": "league",
        "timeout": 500,
    },
    "turn": 14,
    "board": {
        "height": 11,
        "width": 11,
        "food": [{"x": 5, "y": 5}, {"x": 9, "y": 0}, {"x": 2, "y": 6}],
        "hazards": [{"x": 3, "y": 2}],
        "snakes": [
            {
                "id": "snake-508e96ac-94ad-11ea-bb37",
                "name": "My Snake",
                "health": 54,
                "body": [{"x": 0, "y": 0}, {"x": 1, "y": 0}, {"x": 2, "y": 0}],
                "latency": "111",
                "head": {"x": 0, "y": 0},
                "length": 3,
                "shout": "why are we shouting??",
                "squad": "",
                "customizations": {
                    "color": "#FF0000",
                    "head": "pixel",
                    "tail": "pixel",
                },
            },
            {
                "id": "snake-b67f4906-94ae-11ea-bb37",
                "name": "Another Snake",
                "health": 16,
                "body": [
                    {"x": 5, "y": 4},
                    {"x": 5, "y": 3},
                    {"x": 6, "y": 3},
                    {"x": 6, "y": 2},
                ],
                "latency": "222",
                "head": {"x": 5, "y": 4},
                "length": 4,
                "shout": "I'm not really sure...",
                "squad": "",
                "customizations": {
                    "color": "#26CF04",
                    "head": "silly",
                    "tail": "curled",
                },
            },
        ],
    },
    "you": {
        "id": "snake-508e96ac-94ad-11ea-bb37",
        "name": "My Snake",
        "health": 54,
        "body": [{"x": 0, "y": 0}, {"x": 1, "y": 0}, {"x": 2, "y": 0}],
        "latency": "111",
        "head": {"x": 0, "y": 0},
        "length": 3,
        "shout": "why are we shouting??",
        "squad": "",
        "customizations": {
            "color": "#FF0000",
            "head": "pixel",
            "tail": "pixel",
        },
    },
}

In [ ]:
def get_active_features(data, feature_mapping):
    active_features = set()

    my_id = data["you"]["id"]
    my_squad = data["you"]["squad"]
    game_type = data["game"]["ruleset"]["name"]
    board_size = data["board"]["height"]
    foods = data["board"]["food"]
    hazards = data["board"]["hazards"]
    snakes = data["board"]["snakes"]

    for food in foods:
        active_feature = (game_type, board_size, food["x"], food["y"], "food")
        active_features.add(feature_mapping[active_feature])

    for hazard in hazards:
        active_feature = (game_type, board_size, hazard["x"], hazard["y"], "hazard")
        active_features.add(feature_mapping[active_feature])

    for snake in snakes:
        if snake["id"] == my_id:
            player = "you"
        elif game_type == "squad" and snake["squad"] == my_squad:
            player = "squad"
        else:
            player = "snake"

        head = snake["head"]
        x = head["x"]
        y = head["y"]
        health = snake["health"]
        active_feature = (game_type, board_size, x, y, player, "head", health)
        active_features.add(feature_mapping[active_feature])

        for previous_body, body in zip(snake["body"][:-1], snake["body"][1:]):
            previous_x = previous_body["x"]
            previous_y = previous_body["y"]
            x = body["x"]
            y = body["y"]

            if previous_x < x:
                active_feature = (game_type, board_size, x, y, player, "left")
            elif previous_x > x:
                active_feature = (game_type, board_size, x, y, player, "right")
            elif previous_y < y:
                active_feature = (game_type, board_size, x, y, player, "down")
            elif previous_y > y:
                active_feature = (game_type, board_size, x, y, player, "up")
            else:
                active_feature = (game_type, board_size, x, y, player, "bottom")

            active_features.add(feature_mapping[active_feature])

    active_features = tuple(active_features)

    return active_features

def get_feature_mapping():
    game_types = ["constrictor", "royale", "solo", "squad", "standard", "wrapped"]
    board_sizes = [7, 11, 19]
    players = ["you", "squad", "snake", "food", "hazard"]
    player_set = {"you", "squad", "snake"}
    pieces = ["left", "right", "down", "up", "bottom", "head"]
    piece_set = {"head"}
    healths = range(1, 101)

    feature_mapping = {}
    index = 0
    for game_type in game_types:
        for board_size in board_sizes:
            for x in range(board_size):
                for y in range(board_size):
                    for player in players:
                        if player in player_set:
                            for piece in pieces:
                                if piece in piece_set:
                                    for health in healths:
                                        feature = (
                                            game_type,
                                            board_size,
                                            x,
                                            y,
                                            player,
                                            piece,
                                            health,
                                        )
                                        feature_mapping[feature] = index
                                        index += 1
                                else:
                                    feature = (
                                        game_type,
                                        board_size,
                                        x,
                                        y,
                                        player,
                                        piece,
                                    )
                                    feature_mapping[feature] = index
                                    index += 1
                        else:
                            feature = (game_type, board_size, x, y, player)
                            feature_mapping[feature] = index
                            index += 1

    return feature_mapping

feature_mapping = get_feature_mapping()
len(get_active_features(data, feature_mapping)) / len(feature_mapping)

In [ ]:
def get_active_features(data, feature_mapping):
    def get_square(x, y):
        return y * 25 + x
    
    active_features = set()

    my_id = data["you"]["id"]
    my_squad = data["you"]["squad"]
    game_type = data["game"]["ruleset"]["name"]
    board_size = data["board"]["height"]
    foods = data["board"]["food"]
    hazards = data["board"]["hazards"]
    snakes = data["board"]["snakes"]
    
    for x in range(board_size):
        for y in range(board_size):
            active_features.add(feature_mapping[(get_square(x, y), ("height", board_size), None)])
            active_features.add(feature_mapping[(get_square(x, y), ("width", board_size), None)])

    for food in foods:
        active_feature = (get_square(food["x"], food["y"]), "food", None)
        active_features.add(feature_mapping[active_feature])

    for hazard in hazards:
        active_feature = (get_square(hazard["x"], hazard["y"]), "hazard", None)
        active_features.add(feature_mapping[active_feature])

    for snake in snakes:
        if snake["id"] == my_id:
            player = "you"
        elif game_type == "squad" and snake["squad"] == my_squad:
            player = "squad"
        else:
            player = "snake"
            
        health = snake["health"]
        head = snake["head"]
        length = snake["length"]

        active_feature = (get_square(head["x"], head["y"]), ("health", health), player)
        active_features.add(feature_mapping[active_feature])
        
        active_feature = (get_square(head["x"], head["y"]), "head", player)
        active_features.add(feature_mapping[active_feature])
        
        active_feature = (get_square(head["x"], head["y"]), ("length", length), player)
        active_features.add(feature_mapping[active_feature])

        for body in snake["body"][1:]:
            active_feature = (get_square(head["x"], head["y"]), ("health", health), player)
            active_features.add(feature_mapping[active_feature])
            
            active_feature = (get_square(body["x"], body["y"]), "body", player)
            active_features.add(feature_mapping[active_feature])
            
            active_feature = (get_square(head["x"], head["y"]), ("length", length), player)
            active_features.add(feature_mapping[active_feature])
        
        # if game_type == "standard"

    active_features = tuple(active_features)

    return active_features

def get_feature_mapping():
    def get_square(x, y):
        return y * 25 + x
    
    healths = range(1, 100 + 1)
    piece_types = ["body", "head"]
    lengths = range(3, 25 * 25 + 1)
    directions = ["up", "down", "left", "right", "noop"]
    
    colours = ["you", "snake"]
    

    feature_mapping = {}
    index = 0
    for x in range(25):
        for y in range(25):
            square = get_square(x, y)
            
            for health in healths:
                for colour in colours:
                    feature_mapping[(square, ("health", health), colour)] = index
                    index += 1

            for piece_type in piece_types:
                for colour in colours:
                    feature_mapping[(square, piece_type, colour)] = index
                    index += 1

            for length in lengths:
                for colour in colours:
                    feature_mapping[(square, ("length", length), colour)] = index
                    index += 1

            feature_mapping[(square, "food", None)] = index
            index += 1

            feature_mapping[(square, "hazard", None)] = index
            index += 1
            
            for height in range(5, 25 + 1):
                feature_mapping[(square, ("height", height), None)] = index
                index += 1
            
            for width in range(5, 25 + 1):
                feature_mapping[(square, ("width", width), None)] = index
                index += 1
            
            #feature_mapping[(square, "wall", None)] = index
            #index += 1
    
    return feature_mapping

feature_mapping = get_feature_mapping()
len(get_active_features(data, feature_mapping)) / len(feature_mapping) * 100

In [ ]:
len(feature_mapping)

In [ ]:
len(get_active_features(data, feature_mapping))

In [ ]:
from os import listdir
from os.path import isfile

import torch

prefix = 'C:/Users/olive/Documents/GitHub/untimely-neglected-wearable/datasets'
datasets = ['training', 'validation', 'test']
game_types = ["constrictor"]
board_sizes = [7, 11, 19]

length = float('-inf')
state = None
for dataset in datasets:
    for game_type in game_types:
        for board_size in board_sizes:
            for filename in listdir(f'{prefix}/{dataset}/{game_type}/{board_size}'):
                if isfile(f'{prefix}/{dataset}/{game_type}/{board_size}/{filename}'):
                    file = torch.load(f'{prefix}/{dataset}/{game_type}/{board_size}/{filename}')
                    if len(file['states']) > length:
                        length = len(file['states'])
                        state = file['states'][length // 2]

In [6]:
def get_active_features(data, feature_mapping):
    
    my_id = data["you"]["id"]
    foods = data["board"]["food"]
    snakes = data["board"]["snakes"]

    features = {feature_mapping[((food["x"], food["y"]), "food")] for food in foods}

    for snake in snakes:
        color = "you" if snake["id"] == my_id else "snake"
        health = ("health", snake["health"])
        length = ("length", snake["length"])

        head = snake["head"]
        square = (head["x"], head["y"])
        features.add(feature_mapping[(square, health)])
        features.add(feature_mapping[(square, "head")])
        features.add(feature_mapping[(square, length)])
        features.add(feature_mapping[(square, color)])

        for body in snake["body"][1:]:
            square = (body["x"], body["y"])
            features.add(feature_mapping[(square, health)])
            features.add(feature_mapping[(square, "body")])
            features.add(feature_mapping[(square, length)])
            features.add(feature_mapping[(square, color)])

        for previous_body, body, next_body in zip(snake["body"][:-2], snake["body"][1:-1], snake["body"][2:]):
            square = (body["x"], body["y"])

            if previous_body["x"] < body["x"]:
                features.add(feature_mapping[(square, ("previous", "left"))])
            elif previous_body["x"] > body["x"]:
                features.add(feature_mapping[(square, ("previous", "right"))])
            elif previous_body["y"] < body["y"]:
                features.add(feature_mapping[(square, ("previous", "down"))])
            elif previous_body["y"] > body["y"]:
                features.add(feature_mapping[(square, ("previous", "up"))])
            else:
                features.add(feature_mapping[(square, ("previous", "noop"))])

            if next_body["x"] < body["x"]:
                features.add(feature_mapping[(square, ("next", "left"))])
            elif next_body["x"] > body["x"]:
                features.add(feature_mapping[(square, ("next", "right"))])
            elif next_body["y"] < body["y"]:
                features.add(feature_mapping[(square, ("next", "down"))])
            elif next_body["y"] > body["y"]:
                features.add(feature_mapping[(square, ("next", "up"))])
            else:
                features.add(feature_mapping[(square, ("next", "noop"))])

    features = tuple(features)

    return features

def get_feature_mapping():
    healths = range(1, 100 + 1)
    piece_types = ["body", "head"]
    lengths = range(3, 11 * 11 + 1)
    directions = ["up", "down", "left", "right", "noop"]
    colors = ["you", "snake"]

    feature_mapping = {}
    index = 0
    for x in range(11):
        for y in range(11):

            for health in healths:
                feature_mapping[((x, y), ("health", health))] = index
                index += 1

            for piece_type in piece_types:
                feature_mapping[((x, y), piece_type)] = index
                index += 1

            for length in lengths:
                feature_mapping[((x, y), ("length", length))] = index
                index += 1

            for direction in directions:
                feature_mapping[((x, y), ("next", direction))] = index
                index += 1

            for direction in directions:
                feature_mapping[((x, y), ("previous", direction))] = index
                index += 1

            for color in colors:
                feature_mapping[((x, y), color)] = index
                index += 1

            feature_mapping[((x, y), "food")] = index
            index += 1

    return feature_mapping

feature_mapping = get_feature_mapping()
len(get_active_features(data, feature_mapping)) / len(feature_mapping) * 100

0.13067740340467615

In [8]:
len(feature_mapping)

28314

In [11]:
def get_active_features(data, feature_mapping):
    def get_square(x, y):
        return y * 11 + x
    
    active_features = set()
    
    my_id = data["you"]["id"]
    foods = data["board"]["food"]
    snakes = data["board"]["snakes"]

    for food in foods:
        square = get_square(food["x"], food["y"])
        active_feature = (square, "food", None)
        active_features.add(feature_mapping[active_feature])

    for snake in snakes:
        color = "you" if snake["id"] == my_id else "snake"
            
        health = snake["health"]
        head = snake["head"]
        length = snake["length"]
        square = get_square(head["x"], head["y"])
        
        active_feature = (square, ("health", health), color)
        active_features.add(feature_mapping[active_feature])
        
        active_feature = (square, "head", color)
        active_features.add(feature_mapping[active_feature])
        
        active_feature = (square, ("length", length), color)
        active_features.add(feature_mapping[active_feature])

        for body in snake["body"][1:]:
            square = get_square(body["x"], body["y"])
            
            active_feature = (square, ("health", health), color)
            active_features.add(feature_mapping[active_feature])
            
            active_feature = (square, "body", color)
            active_features.add(feature_mapping[active_feature])
            
            active_feature = (square, ("length", length), color)
            active_features.add(feature_mapping[active_feature])
        
        for previous_body, body, next_body in zip(snake["body"][:-2], snake["body"][1:-1], snake["body"][2:]):
            square = get_square(body["x"], body["y"])
            
            if previous_body["x"] < body["x"]:
                active_feature = (square, ("previous", "left"), color)
            elif previous_body["x"] > body["x"]:
                active_feature = (square, ("previous", "right"), color)
            elif previous_body["y"] < body["y"]:
                active_feature = (square, ("previous", "down"), color)
            elif previous_body["y"] > body["y"]:
                active_feature = (square, ("previous", "up"), color)
            else:
                active_feature = (square, ("previous", "noop"), color)
            
            active_features.add(feature_mapping[active_feature])
            
            if next_body["x"] < body["x"]:
                active_feature = (square, ("next", "left"), color)
            elif next_body["x"] > body["x"]:
                active_feature = (square, ("next", "right"), color)
            elif next_body["y"] < body["y"]:
                active_feature = (square, ("next", "down"), color)
            elif next_body["y"] > body["y"]:
                active_feature = (square, ("next", "up"), color)
            else:
                active_feature = (square, ("next", "noop"), color)

            active_features.add(feature_mapping[active_feature])

    active_features = tuple(active_features)

    return active_features

def get_feature_mapping():
    def get_square(x, y):
        return y * 11 + x
    
    healths = range(1, 100 + 1)
    piece_types = ["body", "head"]
    lengths = range(3, 11 * 11 + 1)
    directions = ["up", "down", "left", "right", "noop"]
    colors = ["you", "snake"]
    

    feature_mapping = {}
    index = 0
    for x in range(11):
        for y in range(11):
            square = get_square(x, y)
            
            for health in healths:
                for color in colors:
                    feature_mapping[(square, ("health", health), color)] = index
                    index += 1

            for piece_type in piece_types:
                for color in colors:
                    feature_mapping[(square, piece_type, color)] = index
                    index += 1

            for length in lengths:
                for color in colors:
                    feature_mapping[(square, ("length", length), color)] = index
                    index += 1
            
            for direction in directions:
                for color in colors:
                    feature_mapping[(square, ("next", direction), color)] = index
                    index += 1
            
            for direction in directions:
                for color in colors:
                    feature_mapping[(square, ("previous", direction), color)] = index
                    index += 1

            feature_mapping[(square, "food", None)] = index
            index += 1
    
    return feature_mapping

feature_mapping = get_feature_mapping()
len(get_active_features(data, feature_mapping)) / len(feature_mapping) * 100

0.05354943505346019

In [13]:
len(feature_mapping)

56023